In [249]:
import numpy as np, random, operator 
import pandas as pd
import matplotlib.pyplot as plt
import random
from numpy import loadtxt
arr = loadtxt("kroD100_distMatrix.txt")
CITIES = pd.DataFrame(arr)
print(CITIES.columns)

RangeIndex(start=0, stop=100, step=1)


In [250]:
POP_SIZE = 50

In [251]:
CITIES.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.0,2793.0,2097.0,1655.0,2251.0,2118.0,1515.0,685.0,2586.0,2736.0,...,1297.0,2778.0,1723.0,509.0,3179.0,1639.0,779.0,971.0,1877.0,2719.0
1,2793.0,0.0,993.0,1157.0,725.0,1114.0,3132.0,2871.0,956.0,1101.0,...,1626.0,388.0,1070.0,3302.0,1306.0,1785.0,3565.0,3751.0,2415.0,102.0
2,2097.0,993.0,0.0,572.0,268.0,156.0,2156.0,2004.0,506.0,688.0,...,803.0,776.0,669.0,2581.0,1132.0,798.0,2870.0,2990.0,1426.0,980.0
3,1655.0,1157.0,572.0,0.0,623.0,682.0,2091.0,1723.0,1065.0,1256.0,...,499.0,1126.0,178.0,2160.0,1694.0,894.0,2433.0,2601.0,1583.0,1095.0
4,2251.0,725.0,268.0,623.0,0.0,402.0,2415.0,2219.0,504.0,707.0,...,985.0,543.0,649.0,2747.0,1112.0,1061.0,3029.0,3172.0,1694.0,713.0


In [252]:
CITIES.iloc[39][76]

637.0

In [351]:
class Individual:
    def __init__(self, gnome = None):
        if(gnome is None):
            self.gnome = self.CreateGnome()
        else:
            self.gnome = gnome
        self.fitness = self.CalcFitness()
    def __str__(self):
        temp = self.gnome.replace(" ", "->")
        return "(Gnome: " + temp + ": Fitness: " + str(self.fitness) + ")"
        #return "(Gnome:{} Fitness: {})".format(gnome.replace(" ", "->"), self.fitness)
    def __repr__(self):
        temp = self.gnome.replace(" ", "->")
        return "(Gnome: " + temp + "\n Fitness: " + str(self.fitness) + ")"
        #return "(Gnome:{} Fitness: {})".format(gnome.replace(" ", "->"), self.fitness)
    def CreateGnome(self):
        gnome = ""
        citynumber = len(CITIES)
        # random number list without recurring items
        rndList = random.sample(range(100), 100)
        counter = 0
        for i in rndList:
            temp = str(i)
            gnome += temp + " "
        return gnome.strip()
    def CalcFitness(self):
        f = 0
        temp = self.gnome.split(" ")
        for i in range(len(temp) - 1):
            #print((temp[i])+"->"+temp[i+1])
            #print(CITIES.iloc[int(temp[i])][int(temp[i+1])])
            f += CITIES.iloc[int(temp[i])][int(temp[i+1])]
        return f

In [341]:
class Population:
    def __init__(self, population_size, initialise):
        self.individuals = []

        #Creates the individuals
        if (initialise):
            for i in range(population_size):
                new_individual = Individual()
                self.individuals.append(new_individual)
    def size(self):
        return len(self.individuals)
    def get_individual(self, index):
        return self.individuals[index]
    def get_fitness(self, individual_passed):
        return self.individuals[index].fitness
    def get_fittest(self):
        fittest = self.individuals[0]
        for i in range(len(self.individuals)):
            if self.get_fitness(fittest) <= self.get_fitness(self.individuals[i]) :
                fittest = self.individuals[i]
        return fittest
    def __str__(self):
        temp = ""
        for i in self.individuals:
            temp += i
        return temp
    def printPop(self):
        for i in self.individuals:
            print(i)

In [342]:
def GenerateRandomIndexes(listsize = len(CITIES)):
    arr = random.sample(range(1, listsize), 2)
    lastI = max(arr)
    startI = min(arr)
    return lastI, startI

In [343]:
def InversionMutation(gnome):
    temp = gnome.split(" ")
    lastI,  startI= GenerateRandomIndexes()
    temp[startI:lastI] = temp[startI:lastI][::-1]
    return " ".join(temp)
    

In [356]:
def OrderCrossover(p1,p2, startindex = None, lastindex = None):
    def crossover(pA, pB, i, j):
        parentA = pA.copy()
        parentB = pB.copy()
        parentA = [i for i in parentA if i != 0]
        parentB = [i for i in parentB if i != 0]

        sub1, sub2 = i, j
        # fill with -1
        child = [-1 for i in range(len(parentA))]
        # insert selected subarray to child
        child[sub1: sub2+1] = parentA[sub1:sub2+1]
        #print(child1)
        #print(child2)
        # We remove the items present in the selected parentA subarray from parentB
        for i in parentA[sub1:sub2+1] :
            parentB.remove(i)
        for i in parentB[sub1:sub2+1]:
            parentA.remove(i)
        child[0:sub1] = parentB[0:sub1]
        child[sub2+1:] = parentB[sub1:]
        return " ".join(child)
    if(lastindex == None or startindex == None):
        lastI,  startI= GenerateRandomIndexes(len(p1.gnome))
    else:
        lastI,  startI= lastindex, startindex
    
    pA = p1.gnome.split(" ")
    pB = p2.gnome.split(" ")
    c1 = crossover(pA,pB,startI, lastI)
    c2 = crossover(pB,pA,startI, lastI)
    return c1,c2

In [345]:
def TournamentSelection(population_passed, tournament_size = 2):
    #Tournament pool
    tournament = Population(tournament_size, False)

    """ Tournament selection technique.
       How it works: The algorithm choose randomly five
       individuals from the population and returns the fittest one """
    for i in range(tournament_size):
        random_id = int(random.random() * population_passed.size())
        tournament.individuals.append(population_passed.get_individual(random_id))

    fittest = tournament.get_fittest()
    return fittest
    

## Test Custom Methos

#### todo: create new lib from custom methods and seperate tests to another file

In [325]:
Pop = Population(100, True)

In [326]:
print(Pop.size() + 1)

101


In [327]:
Pop.printPop()

(Gnome: 12->6->88->17->16->74->45->60->99->20->64->1->58->81->34->72->96->32->94->68->39->98->95->3->25->87->59->15->24->35->79->75->30->9->5->73->77->23->22->26->76->2->4->83->69->91->18->61->51->82->44->46->33->71->62->11->13->19->90->65->37->66->0->7->49->78->21->84->55->67->36->27->43->63->80->50->97->86->54->89->10->93->8->28->70->85->29->47->56->14->53->92->52->41->57->42->48->40->38->31: Fitness: 147164.0)
(Gnome: 79->14->18->66->89->3->19->17->46->90->94->35->16->34->0->74->77->11->80->39->31->21->4->54->56->61->82->92->15->76->28->58->70->95->88->50->24->36->83->27->30->55->8->45->98->81->78->63->62->85->33->87->40->12->41->47->93->64->37->13->60->6->5->53->48->65->38->75->86->71->97->49->59->7->96->29->84->9->57->51->26->44->69->23->72->42->91->22->20->67->2->52->1->73->10->68->99->25->43->32: Fitness: 159318.0)
(Gnome: 93->6->48->85->16->71->38->20->47->1->28->17->24->25->91->4->34->26->80->63->61->55->9->3->14->32->12->89->97->21->45->43->0->31->8->70->30->86->72->60->62->4

In [319]:
TournamentSelection(Pop, 10)

(Gnome: 39->45->17->72->87->44->76->83->86->34->65->2->73->95->24->90->56->66->1->46->6->68->12->94->79->69->52->70->49->78->59->18->85->36->42->29->50->40->97->22->58->23->3->92->19->33->77->25->93->27->41->99->32->54->88->57->13->26->4->60->7->84->14->35->98->0->28->62->89->61->31->67->15->38->91->55->80->43->10->71->47->9->63->64->48->11->5->51->96->21->82->81->37->16->20->74->8->75->53->30
 Fitness: 172137.0)

In [14]:
gnomesample = CreateGnome()

In [15]:
gnomesample.replace(" ", "->")

'20->71->84->48->22->44->5->46->87->3->51->18->1->32->10->8->90->78->85->25->98->83->62->36->96->49->28->34->79->2->77->76->9->99->55->24->12->16->58->19->72->43->13->35->89->39->94->23->54->75->15->37->47->29->6->40->70->64->91->82->21->95->26->80->45->7->86->73->81->69->31->65->17->61->88->38->66->74->30->93->11->63->27->68->0->33->92->42->50->14->59->53->4->41->52->57->60->67->97->56'

In [16]:
samplearr = gnomesample.split(" ")
len(samplearr)

100

In [17]:
fitVal = CalcFitness(gnomesample)
print(fitVal)

162273.0


In [19]:
i1 = Individual()
i1

(Gnome: 15->30->55->25->60->21->31->14->52->12->29->51->8->91->85->36->95->28->74->77->97->20->27->6->64->4->41->58->7->50->65->94->76->66->87->83->10->80->24->9->33->82->93->0->56->69->16->84->75->57->47->32->89->44->53->22->98->73->26->71->5->39->59->63->88->92->78->19->96->37->38->70->81->34->11->3->61->72->54->2->1->23->35->99->45->49->68->40->17->13->42->48->67->43->90->18->86->62->46->79
 Fitness: 161876.0)

In [232]:
i2 = Individual()
print(i2)

(Gnome: 83->80->62->28->11->29->10->60->12->32->67->0->49->44->41->2->66->14->50->21->20->39->22->93->58->71->74->45->18->94->75->47->61->65->9->88->46->91->13->87->55->69->40->25->38->99->1->89->3->68->85->73->56->59->5->63->64->70->37->6->96->17->86->77->72->27->82->4->98->78->43->16->15->97->34->57->36->53->26->7->51->90->33->95->81->42->76->19->48->31->24->8->92->79->35->84->54->52->30->23: Fitness: 167824.0)


In [233]:
InversionMutation(i2.gnome)

'83 80 62 28 11 29 10 60 12 32 67 0 49 44 41 2 66 14 50 21 20 13 91 46 88 9 65 61 47 75 94 18 45 74 71 58 93 22 39 87 55 69 40 25 38 99 1 89 3 68 85 73 56 59 5 63 64 70 37 6 96 17 86 77 72 27 82 4 98 78 43 16 15 97 34 57 36 53 26 7 51 90 33 95 81 42 76 19 48 31 24 8 92 79 35 84 54 52 30 23'

In [157]:
# p1        =   4   9   2   8   3   1   5   7   6
# p2        =   6   4   1   3   7   2   8   5   9

# child 1   =   6   4   2   8   3   1   7   5   9
# child 2   =   4   9   1   3   7   2   8   5   6

In [226]:
p1= [4,9,2,8,3,1,5,7,6]
p2= [6,4,1,3,7,2,8,5,9]
startI,  lastI = 2,5

In [227]:
OrderCrossover(p1,p2, startI, lastI)

([6, 4, 2, 8, 3, 1, 7, 5, 9], [4, 9, 1, 3, 7, 2, 8, 5, 6])

## End Test

### create population:

In [352]:
POPULATION = Population(100, True)

In [353]:
POPULATION.printPop()

(Gnome: 87->53->92->82->65->84->16->60->4->49->11->17->67->19->94->57->3->15->9->79->54->96->42->14->38->26->66->83->25->95->37->78->70->77->73->39->64->72->8->31->71->80->23->44->0->86->81->56->63->18->33->1->43->34->59->46->62->69->68->12->76->36->32->30->75->2->97->13->6->40->50->52->20->41->90->51->88->29->61->24->5->58->85->27->98->93->89->91->22->28->35->48->99->10->47->7->21->45->74->55: Fitness: 159331.0)
(Gnome: 55->24->64->72->12->43->34->49->11->82->88->51->93->2->95->99->60->3->9->73->19->79->5->40->35->78->25->83->54->74->48->89->62->44->87->81->91->65->46->27->84->6->75->17->58->92->10->71->69->30->66->0->67->16->96->57->77->68->90->20->52->21->33->42->63->26->15->14->32->39->85->41->38->59->53->37->98->18->50->28->61->8->1->31->13->47->56->94->29->45->4->22->36->97->76->86->23->80->7->70: Fitness: 163031.0)
(Gnome: 35->73->42->54->56->4->60->20->78->74->70->40->30->27->8->45->21->29->49->90->34->88->82->84->37->63->75->36->22->62->0->39->69->26->2->93->81->57->94->64->5-

### evolve

#### with order crossover

In [370]:
new_population = Population(POPULATION.size(), False)
r = int(POPULATION.size()/2)
for i in range(r):
    individual1 = TournamentSelection(POPULATION, tournament_size=5)
    individual2 = TournamentSelection(POPULATION, tournament_size=5)
    new_individual_gnome1, new_individual_gnome2= OrderCrossover(individual1, individual2)
    new_individual1 = Individual(new_individual_gnome1)
    new_individual2 = Individual(new_individual_gnome2)
    new_population.individuals.append(new_individual1)
    new_population.individuals.append(new_individual2)

In [371]:
new_population.printPop()

(Gnome: 20->81->82->24->39->15->29->7->3->78->38->9->43->23->58->34->87->25->66->86->91->48->90->61->8->6->72->94->41->19->56->99->75->69->35->60->54->31->40->73->84->83->1->37->22->18->50->59->76->70->96->53->33->92->68->49->45->62->14->80->77->89->32->30->63->11->47->36->98->64->65->42->52->93->97->85->2->55->5->21->28->67->0->95->46->51->13->71->10->4->79->26->12->88->57->27->16->44->74->17: Fitness: 157921.0)
(Gnome: 99->84->25->86->35->58->80->68->34->73->3->89->60->6->56->92->75->43->96->23->37->62->36->41->31->94->22->32->45->47->24->33->11->65->9->40->66->42->30->19->72->63->50->7->91->20->14->59->76->29->70->38->49->61->64->39->1->53->77->48->81->78->90->82->15->83->8->69->54->87->18->98->51->13->71->10->4->79->26->12->57->27->16->74->17->52->93->88->97->44->85->2->55->5->21->28->67->0->95->46: Fitness: 169614.0)
(Gnome: 84->53->82->11->19->94->16->4->57->60->15->92->65->17->49->87->3->9->67->79->54->96->42->14->38->26->66->83->25->95->37->78->70->77->73->39->64->72->8->31->71

In [372]:
new_population.size()

100

#### with mutation

In [374]:
new_population = Population(POPULATION.size(), False)
for i in range(POPULATION.size()):
    individual1 = TournamentSelection(POPULATION, tournament_size=5)
    new_individual_gnome1= InversionMutation(individual1.gnome)
    new_individual1 = Individual(new_individual_gnome1)
    new_population.individuals.append(new_individual1)

In [375]:
new_population.printPop()

(Gnome: 61->64->5->16->2->63->13->36->57->68->53->66->38->23->80->79->11->7->9->82->18->48->39->37->43->73->29->6->49->17->32->85->34->88->8->99->35->94->12->96->41->1->69->28->42->70->24->74->89->50->20->47->78->54->46->3->45->27->33->14->52->21->90->93->91->22->65->51->76->84->19->83->59->44->97->56->75->55->72->98->67->25->81->40->58->92->30->0->10->77->87->60->26->71->15->62->86->95->4->31: Fitness: 173862.0)
(Gnome: 50->71->51->3->8->42->11->97->6->91->12->33->19->15->85->32->88->30->65->80->98->81->22->45->75->56->36->40->66->18->44->90->9->70->68->49->76->13->48->25->24->27->37->92->60->67->4->52->64->99->59->41->93->28->46->72->7->79->14->86->38->89->78->5->63->21->1->31->53->23->84->95->69->77->29->54->26->62->39->16->94->47->0->43->83->61->10->96->35->55->17->34->82->73->74->57->20->58->2->87: Fitness: 163571.0)
(Gnome: 19->13->78->67->42->12->71->90->35->77->54->72->41->6->91->15->20->80->37->68->56->31->75->84->76->40->50->10->62->4->36->11->23->21->66->17->73->0->74->46->5

In [376]:
new_population.size()

100